In [1]:
import pandas as pd
import numpy as np
import sklearn
from xgboost import XGBClassifier
import xgboost
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from xgboost.sklearn import XGBClassifier
from sklearn import metrics   #Additional scklearn functions
from sklearn.model_selection import cross_validate,GridSearchCV

import pickle
from sklearn.utils import shuffle
import json
import os
from xgboost import XGBClassifier
from matplotlib import pyplot
import matplotlib.pylab as plt
%matplotlib inline
import math

In [2]:
#Don't use datetime, Don't use categorical features, xgboost doesn't do those, first one hot encode
featuresToUse = ['visitor_hist_starrating', 'visitor_hist_adr_usd',
       'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'price_usd', 'promotion_flag',
       'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'comp1_rate', 'comp1_inv',
                 'mean_prop_starrating',
       'mean_prop_review_score', 'mean_prop_brand_bool',
       'mean_prop_location_score1', 'mean_prop_location_score2',   'mean_prop_log_historical_price','cluster__0','cluster__1','cluster__2','cluster__3','cluster__4','cluster__5','diff_distance_orig','diff_price_usd']
labelToPredict = 'booking_bool'
neededVars = ['srch_id','click_bool']
nameOfModel = "models/10mayClickingOverFitTest"

In [3]:
dftrain = pd.read_csv('../data/80PercentTrainWithMeanNoShuffle.csv', delimiter=",",usecols=featuresToUse+[labelToPredict]+neededVars)
dftrain = dftrain.sample(frac=0.2)
Y = dftrain[labelToPredict]

In [ ]:
 'min_child_weight':range(1,12,2),
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)],
 'gamma':[i/10.0 for i in range(0,5)],
 'learning_rate':[i/10.0 for i in range(0,10)]

In [4]:
param_test1 = {
 'max_depth':range(3,15,2),

}



gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate = 0.1, n_estimators=120, max_depth=5,
min_child_weight=1, gamma=0,
objective= 'reg:logistic', nthread=-1, scale_pos_weight=sum(Y==0)/sum(Y==1), seed=12,verbosity=3), 
param_grid = param_test1, scoring='roc_auc',n_jobs=5,iid=False, cv=5,verbose = 100)

gsearch1.fit(dftrain[featuresToUse],dftrain[labelToPredict])
gsearch1.cv_results_

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
Pickling array (shape=(34,), dtype=object).
Memmapping (shape=(793336,), dtype=int64) to new file C:\Users\Rens\AppData\Local\Temp\joblib_memmapping_folder_16112_1888206506\16112-2526386214952-347895ef66b443c9bf2172f83ac186f9.pkl
Memmapping (shape=(17, 793336), dtype=float64) to new file C:\Users\Rens\AppData\Local\Temp\joblib_memmapping_folder_16112_1888206506\16112-2526386214952-47ab1faca4de4adea8af13e9925ba3bb.pkl
Memmapping (shape=(17, 793336), dtype=int64) to new file C:\Users\Rens\AppData\Local\Temp\joblib_memmapping_folder_16112_1888206506\16112-2526386214952-7e12762ea5294b4fa3676c77f15e4da9.pkl
Pickling array (shape=(17,), dtype=object).
Pickling array (shape=(17,), dtype=object).
Pickling array (shape=(17,), dtype=int64).
Pickling array (shape=(17,), dtype=int64).
Memmapping (shape=(793336,), dtype=int64) to new file C:\Users\Rens\AppData\Local\

C:\Users\Rens\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Rens\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Rens\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Rens\Anaconda3\lib\site-packages\sklearn\utils\de

{'mean_fit_time': array([200.70742841]),
 'std_fit_time': array([2.25416205]),
 'mean_score_time': array([1.04560652]),
 'std_score_time': array([0.04248211]),
 'param_max_depth': masked_array(data=[3],
              mask=[False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 3}],
 'split0_test_score': array([0.71623687]),
 'split1_test_score': array([0.72122372]),
 'split2_test_score': array([0.71865162]),
 'split3_test_score': array([0.71843398]),
 'split4_test_score': array([0.72363985]),
 'mean_test_score': array([0.71963721]),
 'std_test_score': array([0.00255028]),
 'rank_test_score': array([1]),
 'split0_train_score': array([0.73069743]),
 'split1_train_score': array([0.73027179]),
 'split2_train_score': array([0.73036022]),
 'split3_train_score': array([0.731136]),
 'split4_train_score': array([0.72962519]),
 'mean_train_score': array([0.73041813]),
 'std_train_score': array([0.00049937])}

In [5]:
gsearch1.cv_results_

C:\Users\Rens\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Rens\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Rens\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\Rens\Anaconda3\lib\site-packages\sklearn\utils\de

{'mean_fit_time': array([200.70742841]),
 'std_fit_time': array([2.25416205]),
 'mean_score_time': array([1.04560652]),
 'std_score_time': array([0.04248211]),
 'param_max_depth': masked_array(data=[3],
              mask=[False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 3}],
 'split0_test_score': array([0.71623687]),
 'split1_test_score': array([0.72122372]),
 'split2_test_score': array([0.71865162]),
 'split3_test_score': array([0.71843398]),
 'split4_test_score': array([0.72363985]),
 'mean_test_score': array([0.71963721]),
 'std_test_score': array([0.00255028]),
 'rank_test_score': array([1]),
 'split0_train_score': array([0.73069743]),
 'split1_train_score': array([0.73027179]),
 'split2_train_score': array([0.73036022]),
 'split3_train_score': array([0.731136]),
 'split4_train_score': array([0.72962519]),
 'mean_train_score': array([0.73041813]),
 'std_train_score': array([0.00049937])}

In [7]:
file = open("cvScores.txt","a+")
 
file.write(str(gsearch1.cv_results_))
 
file.close() 